In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import asyncio
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from mcp_use import MCPAgent, MCPClient

In [3]:
import hud  # For telemetry


In [6]:

# Load environment variables
load_dotenv()

# Create configuration dictionary
config = {
  "mcpServers": {
    "airbnb": {
      "command": "npx",
      "args": ["-y", "@openbnb/mcp-server-airbnb"]
    }
  }
}

# Create MCPClient from configuration dictionary
client = MCPClient.from_dict(config)

# Create LLM
llm = ChatOpenAI(model="gpt-4o")

# Create agent with the client
agent = MCPAgent(llm=llm, client=client, max_steps=30)

# Generate a unique task ID for this run
task_run_id = f"test-run-{int(asyncio.get_event_loop().time())}"





In [8]:

# Run the query inside a trace context
with hud.trace(task_run_id=task_run_id, attributes={"query": "Find the best restaurant in San Francisco"}):
    result = await agent.run(
        "Find the best restaurant in San Francisco",
    )
    print(f"\nResult: {result}")
    
    # Access the telemetry buffer (for debugging purposes)
    from hud.telemetry.context import flush_buffer
    telemetry_data = flush_buffer()
    print(f"\nCollected {len(telemetry_data)} telemetry records")
    
    # Print first few records to see what was captured
    for i, record in enumerate(telemetry_data[:3]):
        print(f"\nRecord {i+1}:")
        print(f"  Type: {record.get('call_type', 'unknown')}")
        print(f"  Status: {record.get('status', 'unknown')}")
        if 'method' in record:
            print(f"  Method: {record['method']}")
        if 'direction' in record:
            print(f"  Direction: {record['direction']}")


2025-05-18 15:23:52,833 - mcp_use - ERROR - Error in StdioConnectionManager task: 
2025-05-18 15:23:52,835 - mcp_use - ERROR - Failed to connect to MCP implementation: 
2025-05-18 15:23:52,835 - mcp_use - ERROR - ❌ Error running query: 


NotImplementedError: 